In [ ]:
import torch
import fairseq

In [ ]:
import pickle
import os
from tqdm.notebook import trange, tqdm

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

In [ ]:
print(torch.hub.list('pytorch/fairseq'))

In [ ]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

In [ ]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels
        
def read_examples_from_file(data_dir, mode, train_examples):
    file_path = os.path.join(data_dir, "{}.txt".format(mode))
    guid_index = 0
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(
                        guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(
                guid="%s-%d".format(mode, guid_index), words=words, labels=labels))
    if mode == 'train':
        if train_examples < 0 or train_examples > len(examples):
            return examples
        else:
            return examples[:train_examples]
    return examples

In [ ]:
examples = read_examples_from_file('../../data/conll2003', 'train', -1)

In [ ]:
sentences = []
for i in range(0, len(examples)):
    sentences.append(' '.join(examples[i].words))

In [ ]:
en2de.cuda()
de2en.cuda()

In [ ]:
def translate_de(start, end, file_name,temperature):
    trans_result = {}
    for idx in tqdm(range(start, end)):
        trans_result[examples[idx].guid] = en2de.translate(de2en.translate(sentences[idx],  sampling = True, temperature = temperature),  sampling = True, temperature = 0.7)
        if idx % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [ ]:
translate_de(0, 24000, '../../data/conll2003/de_temp_0.8.pkl',0.8)